In [1]:
#import findspark
import findspark
findspark.init()

In [2]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark import SparkConf

#from pyspark.ml.feature import Tokenizer, HashingTF, IDF 
#from pyspark.ml.classification import LogisticRegression
#from pyspark.ml import Pipeline
from pyspark.sql.functions import col


from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer, StopWordsRemover
from pyspark.sql.functions import split,explode


In [3]:
conf = SparkConf().setMaster("local[*]").set("spark.executer.memory", "3g")


In [4]:
sc = SparkContext(conf = conf)
spark = SparkSession(sc)
spark = SparkSession(sc).builder.getOrCreate()
spark.sparkContext.setLogLevel("ERROR")



Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/28 02:47:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/11/28 02:47:23 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/11/28 02:47:23 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/11/28 02:47:23 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/11/28 02:47:23 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
22/11/28 02:47:23 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.
22/11/28 02:47:23 WARN Utils: Service 'SparkUI' could not bind on port 4045. Attempting port 4046.
22/11/28 02:47:23 WARN Utils: Service 'SparkUI' could not bind on port 4046. Attempting port 4047.


In [6]:
#1. Clean the dataset

yelp_rev = spark.read.format("csv").option("quote", "\"").option("escape", "\"").option('multiLine', True).option("encoding", "ISO-8859-1").option("header", "true").load("yelp_review.csv")
yelp_rev = yelp_rev.withColumnRenamed("business_id","business_id_rev")
yelp_rev = yelp_rev.withColumnRenamed("stars","stars_rev")
yelp_rev.printSchema()
#yelp_rev.show()



root
 |-- review_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- business_id_rev: string (nullable = true)
 |-- stars_rev: string (nullable = true)
 |-- date: string (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: string (nullable = true)
 |-- funny: string (nullable = true)
 |-- cool: string (nullable = true)

+--------------------+--------------------+--------------------+---------+----------+--------------------+------+-----+----+
|           review_id|             user_id|     business_id_rev|stars_rev|      date|                text|useful|funny|cool|
+--------------------+--------------------+--------------------+---------+----------+--------------------+------+-----+----+
|vkVSCC7xljjrAI4UG...|bv2nCi5Qv5vroFiqK...|AEx2SYEUJmTxVVB18...|        5|2016-05-28|Super simple plac...|     0|    0|   0|
|n6QzIUObkYshz4dz2...|bv2nCi5Qv5vroFiqK...|VR6GpWIda3SfvPC-l...|        5|2016-05-28|Small unassuming ...|     0|    0|   0|
|MV3CcKScW05u5LVfF

In [8]:
yelp_biz = spark.read.format("csv").option("quote", "\"").option("escape", "\"").option('multiLine', True).option("encoding", "ISO-8859-1").option("header", "true").load("yelp_business.csv")
yelp_biz = yelp_biz.withColumnRenamed("business_id","business_id_biz")
yelp_biz = yelp_biz.withColumnRenamed("stars","stars_biz")
yelp_biz.printSchema()
#yelp_biz.show()

root
 |-- business_id_biz: string (nullable = true)
 |-- name: string (nullable = true)
 |-- neighborhood: string (nullable = true)
 |-- address: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- postal_code: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- stars_biz: string (nullable = true)
 |-- review_count: string (nullable = true)
 |-- is_open: string (nullable = true)
 |-- categories: string (nullable = true)

+--------------------+--------------------+------------------+--------------------+--------------+-----+-----------+-------------+--------------+---------+------------+-------+--------------------+
|     business_id_biz|                name|      neighborhood|             address|          city|state|postal_code|     latitude|     longitude|stars_biz|review_count|is_open|          categories|
+--------------------+--------------------+------------------+-------

In [9]:
biz_rev = yelp_rev.join(yelp_biz,yelp_biz.business_id_biz ==  yelp_rev.business_id_rev,"inner")#.show(truncate=False)
biz_rev.printSchema()



root
 |-- review_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- business_id_rev: string (nullable = true)
 |-- stars_rev: string (nullable = true)
 |-- date: string (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: string (nullable = true)
 |-- funny: string (nullable = true)
 |-- cool: string (nullable = true)
 |-- business_id_biz: string (nullable = true)
 |-- name: string (nullable = true)
 |-- neighborhood: string (nullable = true)
 |-- address: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- postal_code: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- stars_biz: string (nullable = true)
 |-- review_count: string (nullable = true)
 |-- is_open: string (nullable = true)
 |-- categories: string (nullable = true)



In [11]:
biz_rev.createOrReplaceTempView("final_df")
fin = spark.sql("select * from final_df where upper(name) like '%RESTAURANT%'")
fin = fin.select("review_id","user_id","business_id_rev","stars_rev","text","useful","funny","cool","categories","state","city","categories")
fin1 =  fin.withColumn("stars_rev_double", fin["stars_rev"].cast("double"))
fin1.show()
print("************************\n")



+--------------------+--------------------+--------------------+---------+--------------------+------+-----+----+--------------------+-----+---------+--------------------+----------------+
|           review_id|             user_id|     business_id_rev|stars_rev|                text|useful|funny|cool|          categories|state|     city|          categories|stars_rev_double|
+--------------------+--------------------+--------------------+---------+--------------------+------+-----+----+--------------------+-----+---------+--------------------+----------------+
|PIsUSmvaUWB00qv5K...|u0LXt3Uea_GidxRW1...|z8oIoCT1cXz7gZP5G...|        4|This is currently...|     1|    0|   0|Dim Sum;Restauran...|   ON|  Toronto|Dim Sum;Restauran...|             4.0|
|23eqwlZzCWZkADWfd...|u0LXt3Uea_GidxRW1...|Aov96CM4FZAXeZvKt...|        5|This place is awe...|     2|    0|   1|Imported Food;Foo...|   ON|  Toronto|Imported Food;Foo...|             5.0|
|Tv-_7d1sa-6cPTZ20...|u0LXt3Uea_GidxRW1...|7Uti5EeAwm3d

In [13]:


print("************ Popular Business Cities **************") #businesses getting lot of reviews
location = yelp_biz.select('business_id_biz','city')
review = yelp_rev.select('business_id_rev')
merge_city = location.join(review,review.business_id_rev == location.business_id_biz,'inner')
grouped_review_city = merge_city.groupby('city').count()
most_reviewed_city = grouped_review_city.groupby('city').sum()
most_reviewed_city.sort('sum(count)',ascending=False).show(10)
most_reviewed_city

print("************************\n")



************ Popular Business Cities **************


+----------+----------+
|      city|sum(count)|
+----------+----------+
| Las Vegas|   1604227|
|   Phoenix|    576700|
|   Toronto|    430985|
|Scottsdale|    308583|
| Charlotte|    237308|
|Pittsburgh|    179523|
| Henderson|    166868|
|     Tempe|    162782|
|      Mesa|    130886|
| MontrÃ©al|    122613|
+----------+----------+
only showing top 10 rows

************************



In [14]:
print("************ Top Businesses with detailed Reviews **************") 
location = yelp_biz.select('business_id_biz','categories')
review = yelp_rev.select('business_id_rev','text')
merge_city = location.join(review,review.business_id_rev == location.business_id_biz,'inner')
len_rev = merge_city.select(explode(split('categories', ',')).alias('category'),'text') # merge_city.explode(split('categories', ',')).alias('category')
len_rev = len_rev.withColumn("rev_length",length(col("text")))
avg = len_rev.groupBy("category").avg("rev_length")
avg.sort('avg(rev_length)',ascending=False).show(10)

print("************************\n")



************ Top Businesses with detailed Reviews **************


+--------------------+------------------+
|            category|   avg(rev_length)|
+--------------------+------------------+
|Bridal;Jewelry;Sh...|            4231.0|
|Real Estate;Unive...|3394.3333333333335|
|Hotels & Travel;H...|3360.3333333333335|
|Educational Servi...|            3357.0|
|Beauty & Spas;Nai...|            3084.5|
|Restaurants;Event...|3019.3333333333335|
|Party Equipment R...|2934.3333333333335|
|Shopping;Local Se...|2923.6666666666665|
|Automotive;Real E...|           2906.75|
|Retirement Homes;...|2886.3333333333335|
+--------------------+------------------+
only showing top 10 rows

************************



In [15]:
print("************ Businesses with less detailed Reviews **************") 

location = yelp_biz.select('business_id_biz','categories')
review = yelp_rev.select('business_id_rev','text')
merge_city = location.join(review,review.business_id_rev == location.business_id_biz,'inner')
len_rev = merge_city.select(explode(split('categories', ',')).alias('category'),'text') # merge_city.explode(split('categories', ',')).alias('category')
len_rev = len_rev.withColumn("rev_length",length(col("text")))
avg = len_rev.groupBy("category").avg("rev_length")
avg.sort('avg(rev_length)',ascending=True).show(10)

print("************************\n")






************ Businesses with less detailed Reviews **************


+--------------------+------------------+
|            category|   avg(rev_length)|
+--------------------+------------------+
|Food Delivery Ser...|49.333333333333336|
|Local Services;Pe...| 85.66666666666667|
|Fire Departments;...| 86.66666666666667|
|Venues & Event Sp...| 87.66666666666667|
|Active Life;Marti...| 88.33333333333333|
|Grocery;Food;Shop...|              90.0|
|Auto Insurance;Ho...|              93.0|
|Chicken Shop;Rest...|              96.0|
|Active Life;Sport...| 97.33333333333333|
|Laundry Services;...|             101.0|
+--------------------+------------------+
only showing top 10 rows

************************



In [16]:
print("************ 10 highly rated Restraunts **************")

review_star_three = fin1.filter('stars_rev_double >3')
grouped_review = review_star_three.groupby('business_id_rev').count()
review_sort = grouped_review.sort('count',ascending=False)
review_sort.show()
print("************************\n")


************ 10 highly rated Restraunts **************


+--------------------+-----+
|     business_id_rev|count|
+--------------------+-----+
|XXW_OFaYQkkGOGniu...| 2012|
|A-uZAD4zP3rRxb44W...| 1201|
|z6-reuC5BYf_Rth9g...|  957|
|VsewHMsfj1Mgsl2i_...|  868|
|6tSvz_21BMo3a4GaI...|  809|
|DbEszO3wk1xVmN3pC...|  718|
|Os1n1_idfw9vv9kwU...|  663|
|0_aeYE2-VbsZts_Up...|  645|
|DESv2ys6SjBKA4SyD...|  592|
|1ZnVfS-qP19upP_fw...|  576|
|-Ht7HiGBox8lS1Y8I...|  571|
|iA8Ve2sZKN5Vz3mYK...|  546|
|3Gt3xskppi9jZuTrw...|  544|
|e41TP5cXZqSrz50xC...|  534|
|FNe5PPA9pyj8FjcDe...|  533|
|-WLrZPzjKfrftLWaC...|  515|
|XC06xxb7FFYsXcI2E...|  487|
|pik4ebdFa0UcUhnNk...|  463|
|-av1lZI1JDY_RZN2e...|  449|
|wKA2y-F_fPcBDOkpz...|  444|
+--------------------+-----+
only showing top 20 rows

************************



In [17]:
print("************ Restaurant Distribution **************")

from pyspark.sql.functions import split,explode
category = fin1.select('categories')
restaurant_cat = category.select(explode(split('categories', ',')).alias('category'))
group_category = restaurant_cat.groupby('category').count()
top_category = group_category.sort('count',ascending=False)
top_category.show(10,truncate=False)
print("************************\n")


************ Restaurant Distribution **************


+----------------------------------------------------------------------------+-----+
|category                                                                    |count|
+----------------------------------------------------------------------------+-----+
|Chinese;Restaurants                                                         |5478 |
|Restaurants;Mexican                                                         |5240 |
|Restaurants;Chinese                                                         |4760 |
|Mexican;Restaurants                                                         |4521 |
|Restaurants;Thai                                                            |3816 |
|Bars;American (Traditional);Lounges;Nightlife;Breakfast & Brunch;Restaurants|2700 |
|Thai;Restaurants                                                            |2427 |
|Italian;Restaurants                                                         |2330 |
|Restaurants;Vietnamese                                          